![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [46]:
import pandas as pd
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, sep="#")
features.columns = ['nome_var']
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [47]:
%%time
# Inicializar o classificador de árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Medir o tempo de treinamento
start_time = time.time()
clf.fit(X_train, y_train)
training_time = time.time() - start_time

# Avaliar a acurácia na base de treino
train_accuracy = accuracy_score(y_train, clf.predict(X_train))

# Avaliar a acurácia na base de teste
test_accuracy = accuracy_score(y_test, clf.predict(X_test))

print(f"Treinamento - Tempo: {training_time:.4f}s, Acurácia Treino: {train_accuracy:.4f}, Acurácia Teste: {test_accuracy:.4f}")

Treinamento - Tempo: 4.0778s, Acurácia Treino: 0.9758, Acurácia Teste: 0.8802
CPU times: total: 4.05 s
Wall time: 4.11 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [48]:
%%time
# Inicializar o PCA com 1 componente
pca = PCA(n_components=1)

# Aplicar o PCA nos dados de treino e teste
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Inicializar o classificador de árvore de decisão
clf_pca = DecisionTreeClassifier(ccp_alpha=0.001)

# Medir o tempo de treinamento
start_time = time.time()
clf_pca.fit(X_train_pca, y_train)
training_time_pca = time.time() - start_time

# Avaliar a acurácia na base de treino
train_accuracy_pca = accuracy_score(y_train, clf_pca.predict(X_train_pca))

# Avaliar a acurácia na base de teste
test_accuracy_pca = accuracy_score(y_test, clf_pca.predict(X_test_pca))

print(f"PCA 1 Componente - Tempo: {training_time_pca:.4f}s, Acurácia Treino: {train_accuracy_pca:.4f}, Acurácia Teste: {test_accuracy_pca:.4f}")

PCA 1 Componente - Tempo: 0.0396s, Acurácia Treino: 0.4997, Acurácia Teste: 0.4571
CPU times: total: 562 ms
Wall time: 134 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [49]:
h = list(range(1, 10))
h

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [50]:
%%time
# components = list(range(1, 150))
components = [1, 2, 5, 10, 50, 75, 100, 150]
results = []

for n in components:
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    clf_pca = DecisionTreeClassifier(ccp_alpha=0.001)
    
    start_time = time.time()
    clf_pca.fit(X_train_pca, y_train)
    training_time_pca = time.time() - start_time
    
    train_accuracy_pca = accuracy_score(y_train, clf_pca.predict(X_train_pca))
    test_accuracy_pca = accuracy_score(y_test, clf_pca.predict(X_test_pca))
    
    results.append({
        'n_components': n,
        'training_time': training_time_pca,
        'train_accuracy': train_accuracy_pca,
        'test_accuracy': test_accuracy_pca
    })

# Exibir resultados
for result in results:
    print(f"{result['n_components']} Componentes - Tempo: {result['training_time']:.4f}s, Acurácia Treino: {result['train_accuracy']:.4f}, Acurácia Teste: {result['test_accuracy']:.4f}")

1 Componentes - Tempo: 0.0382s, Acurácia Treino: 0.4997, Acurácia Teste: 0.4571
2 Componentes - Tempo: 0.0297s, Acurácia Treino: 0.6128, Acurácia Teste: 0.5847
5 Componentes - Tempo: 0.0407s, Acurácia Treino: 0.8460, Acurácia Teste: 0.7886
10 Componentes - Tempo: 0.0842s, Acurácia Treino: 0.8927, Acurácia Teste: 0.8242
50 Componentes - Tempo: 0.4541s, Acurácia Treino: 0.9154, Acurácia Teste: 0.8283
75 Componentes - Tempo: 0.6879s, Acurácia Treino: 0.9215, Acurácia Teste: 0.8303
100 Componentes - Tempo: 0.8892s, Acurácia Treino: 0.9268, Acurácia Teste: 0.8232
150 Componentes - Tempo: 1.4315s, Acurácia Treino: 0.9270, Acurácia Teste: 0.8286
CPU times: total: 13.1 s
Wall time: 4.78 s


In [54]:
results = pd.DataFrame(results)
results.to_clipboard()

In [52]:
display(results[results['train_accuracy'] == float(results['train_accuracy'].max())])
display(results[results['test_accuracy'] == float(results['test_accuracy'].max())])

,n_components,training_time,train_accuracy,test_accuracy
7,150,1.43148,0.926959,0.828639


,n_components,training_time,train_accuracy,test_accuracy
5,75,0.687856,0.921518,0.830336


In [53]:
import plotly.express as px

df = results
fig = px.line(df, x='n_components', y=['train_accuracy', 'test_accuracy'], title='Acurácia vs Número de Componentes')

fig.update_layout(
    xaxis_title='Número de Componentes',
    yaxis_title='Acurácia',
    legend_title='Tipo de Acurácia',
    plot_bgcolor="white"
)

fig.show()

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### Acurácia

- **Acurácia de treinamento**:
    - Aumenta consistentemente à medida que o número de componentes aumenta, o que é esperado, pois modelos mais complexos geralmente se ajustam melhor aos dados de treinamento.
    - Vai de 0.499 para 0.927, mostrando uma melhoria contínua.
- **Acurácia de teste**:
    - Aumenta de 0.457 para 0.830 com 75 componentes, indicando uma melhoria na capacidade do modelo de generalizar para novos dados.
    - No entanto, observa-se uma ligeira queda com 100 componentes (0.823), sugerindo um possível overfitting, onde o modelo se ajusta muito bem aos dados de treinamento, mas não se generaliza bem para dados de teste.
    - Volta a aumentar ligeiramente para 150 componentes, mas essa melhoria não é muito significativa comparada aos componentes anteriores.

### Tempo de Processamento

- **Tempo de treinamento**:
    - O tempo de processamento aumenta exponencialmente com o número de componentes.
    - Vai de aproximadamente 0.038 segundos para 1 componente, até aproximadamente 1.431 segundos para 150 componentes.
    - Isso reflete a complexidade crescente do modelo à medida que mais componentes são adicionados, o que exige mais recursos computacionais e tempo para treinar.

### Conclusão

- **Acurácia**:
    - A acurácia de treinamento aumenta consistentemente, mas a acurácia de teste mostra sinais de overfitting além de certo ponto (por volta de 100 componentes).
- **Tempo de Processamento**:
    - O tempo de processamento aumenta significativamente com mais componentes, o que pode se tornar um problema prático em termos de custo computacional e tempo necessário para treinar o modelo.